In [23]:
import pandas as pd
import sys

In [24]:
rotg=[pd.read_fwf('../data/'+str(year)+'.txt') for year in range(2000,2016)]

In [25]:
test=rotg[0]
df = pd.DataFrame(test.NAME.str.split(' ',1).tolist(),
                                   columns = ['FIRST_NAME','LAST_NAME'])

print test.columns
test=test.drop('NAME',1)
test=test.join(df)

test.head()



Index([u'PLACE', u'NAME', u'AGE', u'SEX', u'CITY', u'ST', u'TIME', u'PACE'], dtype='object')


,PLACE,AGE,SEX,CITY,ST,TIME,PACE,FIRST_NAME,LAST_NAME
0,1,22,M,Albany,NY,19:52,4:58,Jeffrey,Brooks
1,2,24,M,Albany,NY,19:56,4:59,Nick,Conway
2,3,23,M,Schenectady,NY,20:22,5:06,Peter,Flynn
3,4,19,M,Albany,NY,20:33,5:09,Jaime,Rodriguez
4,5,17,M,West Sand Lake,NY,21:08,5:17,Tyson,Evensen


In [26]:
master=pd.DataFrame()
rotg2=[]
i=0 
year=2000

for i in range(len(rotg)):
    
    try :
        # handle NA
        rotg[i].NAME=rotg[i].NAME.fillna('No Name')
        rotg[i].ST=rotg[i].ST.fillna('NY')
        rotg[i].CITY=rotg[i].CITY.fillna('Albany')
        rotg[i].SEX=rotg[i].SEX.fillna('F')
        rotg[i].TIME=rotg[i].TIME.fillna('59:21.5')
        
        temp = pd.DataFrame(rotg[i].NAME.str.split(' ',1).tolist(),
                                   columns = ['FIRSTNAME','LASTNAME'])
        

        rotg[i]=rotg[i].drop('NAME',1)
        rotg[i]=rotg[i].join(temp)
        
    except :
        print "Unexpected error:", sys.exc_info()[0]
        pass
    newdf=rotg[i][['PLACE', 'FIRSTNAME', 'LASTNAME', 'AGE', 'SEX', 'CITY', 'ST', 'TIME']]
    newdf['YEAR']=[year]*len(newdf.PLACE)
    year=year+1
    master=master.append(newdf)
    i=i+1
    

Unexpected error: <type 'exceptions.AttributeError'>


/home/bnorthan/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
master.head()

,PLACE,FIRSTNAME,LASTNAME,AGE,SEX,CITY,ST,TIME,YEAR
0,1,Jeffrey,Brooks,22,M,Albany,NY,19:52,2000
1,2,Nick,Conway,24,M,Albany,NY,19:56,2000
2,3,Peter,Flynn,23,M,Schenectady,NY,20:22,2000
3,4,Jaime,Rodriguez,19,M,Albany,NY,20:33,2000
4,5,Tyson,Evensen,17,M,West Sand Lake,NY,21:08,2000


In [28]:
master=master.reset_index(drop=True)

In [29]:
# hack?? erase number 213??
master=master.drop(213)
master=master.reset_index(drop=True)

In [30]:
master.to_csv('../output/temp.csv', index=False)

In [9]:
# change times to timestamp
new_time=[]
for i in range(len(master.TIME)):
    if len(master.TIME[i].split(':'))==2:
        time_string='00:'+master.TIME[i]
    else:
        time_string=master.TIME[i]
        
    if (type(pd.to_datetime(time_string))!=pd.tslib.Timestamp):
        time_stamp=pd.to_datetime('00:40:00')
    else:
        time_stamp=pd.to_datetime(time_string)
    
    new_time.append(time_stamp)

master['TIMESTAMP']=new_time

In [11]:
# sanity check --> look at min and max of all fields

print master.TIMESTAMP.min()
print master.TIMESTAMP.max()

print master.AGE.min()
print master.AGE.max()

print master.TIME.min()
print master.TIME.max()

print master.FIRSTNAME.min()
print master.FIRSTNAME.max()

print master.LASTNAME.min()
print master.LASTNAME.max()

print master.ST.min()
print master.ST.max()

print master.CITY.min()
print master.CITY.max()

print master.YEAR.min()
print master.YEAR.max()

print master.PLACE.min()
print master.PLACE.max()

print master.SEX.min()
print master.SEX.max()


#master=master.drop('ST',1)

2016-03-17 00:19:36
2016-03-17 01:05:22
6.0
85.0
19:36
59:21.5
3
susan
A Parente
von Maltzahn
3 Ray Lee Jr.
susan finlayson
314 Princetown
wynantskill
2000
2015
1
663
F
M


In [12]:
# sort by time
master=master.sort('TIMESTAMP')
master.head()

,PLACE,FIRSTNAME,LASTNAME,AGE,SEX,CITY,ST,TIME,YEAR,TIMESTAMP
756,1,Chris,Hartshorn,31,M,Scenectady,Chris Hartshorn,19:36,2003,2016-03-17 00:19:36
2761,1,Andy,Allstadt,26,M,Albany,Andy Allstadt,19:37,2009,2016-03-17 00:19:37
757,2,Jamie,Rodriguez,21,M,Albany,Jamie Rodriguez,19:43,2003,2016-03-17 00:19:43
2762,2,Chuck,Terry,26,M,Albany,Chuck Terry,19:45,2009,2016-03-17 00:19:45
1588,1,Nick,Conway,30,M,Albany,Nick Conway,19:45,2006,2016-03-17 00:19:45


In [13]:
import numpy as np
from bokeh.models import HoverTool
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.models import ColumnDataSource, HoverTool, HBox, VBoxForm
from bokeh.io import curdoc

from bokeh.plotting import Figure, ColumnDataSource, show, output_file

output_file('alltime.html',"All Time ROTG Finishers")

min_year = Slider(title="Start Year", start=2000, end=2015, value=2000, step=1)
max_year = Slider(title="End Year", start=2000, end=2015, value=2015, step=1)

master["color"] = np.where(master["SEX"] == 'F', "red", "blue")

p = Figure(plot_width=1400, plot_height=800, y_axis_type='datetime')

source = ColumnDataSource(data=master)
#source=ColumnDataSource(data=master)

s1=p.scatter('AGE', 'TIMESTAMP', source =  source, color=master.color, size=15, alpha=0.5)

hover = HoverTool()
hover.tooltips = [('First Name', '@FIRSTNAME'), ('Last Name', '@LASTNAME'), ('Time', '@TIME'), ('Year', '@YEAR')]
p.add_tools(hover)

p.title="Runnin' of the Green All Time Finishers"
p.xaxis.axis_label="Age"
p.yaxis.axis_label="Time"

#controls = [min_year, max_year]
#for control in controls:
#    control.on_change('value', update)

    
#inputs = HBox(VBoxForm(*controls), width=300)
#update(None, None, None) # initial load of the data
#curdoc().add_root(inputs)
#curdoc().add_root(HBox(inputs, p, width=1100))
#curdoc().add_root(p)

show(p)

In [70]:
### Scraps --> Trying for interactive plot --> revisit

def select_runners():
    #genre_val = genre.value
    #director_val = director.value.strip()
    #cast_val = cast.value.strip()
    selected = master[
        (master.YEAR >= min_year.value) &
        (master.YEAR <= max_year.value) 
    ]
    
    '''
    if (genre_val != "All"):
        selected = selected[selected.Genre.str.contains(genre_val)==True]
    if (director_val != ""):
        selected = selected[selected.Director.str.contains(director_val)==True]
    if (cast_val != ""):
        selected = selected[selected.Cast.str.contains(cast_val)==True]
    '''
    return selected
    

In [71]:
### Scraps --> Trying for interactive plot --> revisit

def update(attrname, old, new):
    df = select_runners()
    
    source.data = dict(
        x=df['AGE'],
        y=df['TIMESTAMP'],
        year=df["YEAR"])
        
    

In [ ]:
### Scraps --> Trying for interactive plot --> revisit

import numpy as np
from bokeh.models import HoverTool
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.models import ColumnDataSource, HoverTool, HBox, VBoxForm
from bokeh.io import curdoc

from bokeh.plotting import Figure, ColumnDataSource, show, output_file

output_file('best.html',"test")

min_year = Slider(title="Year released", start=2000, end=2015, value=2000, step=1)
max_year = Slider(title="End Year released", start=2000, end=2015, value=2015, step=1)

p = Figure(plot_width=1400, plot_height=800)

x=[1,2,3]
y=[1,2,3]

s1=p.circle(x, y)

curdoc().add_root(p)

#show(p)